# Вебинар "Как я перестал беспокоиться и полюбил Пуассон-Bootstrap" (канал karpov.courses, автор Валерий Бабушкин)

In [1]:
import os
import numpy as np
from typing import Tuple, List
import multiprocessing as mp
from tqdm.notebook import tqdm

### Тест

In [2]:
B = 10
N = 2

In [3]:
def poisson_b(B):
  sampled = np.random.poisson(1,B)
  return sampled

In [4]:
def binomial_b(N):
  list_sampled = []
  for _ in range(B):
    sampled = np.random.binomial(N,1./N)
    list_sampled.append(sampled)
  return list_sampled

In [5]:
%%time
l1 = list(map(poisson_b,[B for _ in range(N)]))

CPU times: user 0 ns, sys: 241 µs, total: 241 µs
Wall time: 249 µs


In [6]:
print(l1)

[array([1, 1, 2, 0, 1, 1, 0, 0, 1, 0]), array([1, 0, 1, 2, 0, 0, 1, 2, 1, 0])]


In [7]:
%%time
l2 = list(map(binomial_b,[N for _ in range(N)]))

CPU times: user 211 µs, sys: 0 ns, total: 211 µs
Wall time: 217 µs


In [8]:
print(l2)

[[2, 1, 0, 1, 1, 0, 1, 2, 1, 0], [1, 1, 2, 2, 1, 1, 0, 0, 2, 2]]


### Пример из вебинара

In [9]:
N_bootstraps: int = 100
def poisson_bootstrap_tp_fp_fn_tn(
    bundle: Tuple[float, List[Tuple[float, float, float, int]]],
                                 ) ->List[np.ndarray]:
    threshold, data = bundle
    TP = np.zeros((N_bootstraps))
    FP = np.zeros((N_bootstraps))
    FN = np.zeros((N_bootstraps))
    TN = np.zeros((N_bootstraps))
    for current_label, current_predict, weight, index in data:
        np.random.seed(index)
        current_predict += np.random.normal(0,0.0125,1)
        current_predict = int(np.clip(current_predict,0,1) >= threshold)
        p_sample = np.random.poisson(1, N_bootstraps) * weight
        
        if current_label == 1 and current_predict == 1:
            TP += p_sample
        if current_label == 1 and current_predict == 0:
            FN += p_sample
        if current_label == 0 and current_predict == 1:
            FP += p_sample
        if current_label == 0 and current_predict == 0:
            TN += p_sample
            
    return [TP, FP, FN, TN]

In [10]:
N = 100
labels = np.random.randint(0,2,N)
predicts = np.clip(np.random.normal(0.5,1,N),0,1)
weights = np.array([1 for _ in range(N)])

print(labels[:10])
print(predicts[:10])
print(weights[:10])

[1 1 0 0 1 1 0 1 0 1]
[0.         0.75077398 0.14852907 1.         0.         0.73624778
 0.         0.55996524 0.         1.        ]
[1 1 1 1 1 1 1 1 1 1]


In [11]:
chunk_size = 10
threshold = 0.81
generator = (
    (
        threshold,
        [
            (labels[x + y],
             predicts[x + y],
             weights[x + y],
             x + y,
            )
    
        for x in range(chunk_size)
        if x+y < N
        ],
        
    )
        for y in range(0,N,chunk_size)

)

In [12]:
cpu_to_use = np.max([os.cpu_count()-1,1])
print(cpu_to_use)

with mp.Pool(processes=cpu_to_use) as pool:
    stat_list = list(tqdm(pool.imap(poisson_bootstrap_tp_fp_fn_tn, generator),
                    total = N//chunk_size))

TP, FP, FN, TN = np.sum(stat_list,0)
print(TP,FP,FN,TN)

1


  0%|          | 0/10 [00:00<?, ?it/s]

[31. 28. 30. 34. 23. 24. 25. 23. 34. 28. 21. 31. 42. 28. 37. 18. 25. 31.
 27. 26. 29. 23. 29. 28. 22. 39. 23. 25. 26. 30. 32. 28. 24. 24. 16. 20.
 28. 26. 26. 33. 21. 12. 31. 21. 25. 30. 24. 21. 24. 26. 21. 26. 20. 34.
 28. 22. 30. 27. 20. 23. 28. 28. 32. 24. 16. 24. 42. 24. 30. 32. 27. 31.
 29. 23. 23. 29. 26. 30. 26. 29. 26. 40. 25. 25. 21. 30. 27. 33. 36. 32.
 34. 32. 34. 30. 26. 31. 20. 29. 24. 28.] [18. 17. 19. 12. 11. 19. 12. 14. 20. 16. 21. 13. 11. 19. 10. 23. 12. 14.
 12. 23. 15. 17. 16. 20. 12. 17. 19. 15. 17. 23. 16. 16. 12. 18. 16. 16.
 15. 22. 17. 13. 24. 15. 13. 15. 13. 25. 14.  8. 16. 21. 10. 18. 14. 19.
 14. 15. 20. 20. 15. 20. 16. 17. 11. 11. 19. 25. 14. 17. 14. 26.  8. 20.
 19. 10. 19. 21. 20. 13. 16. 18. 18. 16. 19. 14. 13. 11. 22. 11. 17. 13.
 16. 13. 18. 15. 12. 18. 16. 13. 13. 16.] [16. 17. 20. 19. 24. 21. 15. 21. 22. 20. 19. 18. 19. 26. 26. 26. 21. 15.
 27. 14. 24. 19. 18. 17. 20. 23. 27. 23. 17. 24. 37. 29. 19. 18. 27. 27.
 21. 27. 17. 28. 31. 22. 35. 31. 27. 26.